In [2]:
from share import *
import config

import cv2
import einops
import gradio as gr
import numpy as np
import torch

import random

from pytorch_lightning import seed_everything
from annotator.util import resize_image, HWC3
from annotator.uniformer import UniformerDetector
from annotator.canny import CannyDetector
from cldm.model import create_model, load_state_dict
from ldm.models.diffusion.ddim import DDIMSampler


apply_canny = CannyDetector()
apply_uniformer = UniformerDetector()

model = create_model('./models/mcldm_v15.yaml').cpu()
# model.load_state_dict(load_state_dict('./models/control_sd15_seg.pth', location='cuda'))
# model = model.cuda()
# ddim_sampler = DDIMSampler(model)

logging improved.
Use Checkpoint: False
Checkpoint Number: [0, 0, 0, 0]
Use global window for all blocks in stage3
load checkpoint from local path: /home/hu/MMIB/controlnet/annotator/ckpts/upernet_global_small.pth
No module 'xformers'. Proceeding without it.
ControlLDM: Running in eps-prediction mode


/home/hu/miniconda3/envs/th/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v1.10.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loaded model config from [./models/mcldm_v15.yaml]


In [3]:
sd1 = torch.load('./models/control_sd15_seg.pth', map_location='cpu')
sd2 = torch.load('./models/control_sd15_canny.pth', map_location='cpu')

In [4]:
new_sd = {}
for k,v in sd1.items():
    if 'control_model' in k:
        new_sd['control_model_1'+k.split('control_model')[1]] = v
    else:
        new_sd[k] = v

for k,v in sd2.items():
    if 'control_model' in k:
        new_sd['control_model_2'+k.split('control_model')[1]] = v


model.load_state_dict(new_sd)

<All keys matched successfully>

In [5]:
from PIL import Image
import numpy as np

input_image = np.array(Image.open('/home/hu/MMIB/remove/sflckr_images/alaska_lakes/43259216952_59352d7204_b.jpg').resize((512,512)))
input_image_1  = np.array(Image.open('/home/hu/MMIB/controlnet/test_imgs/pose2.png').resize((512,512)))

In [6]:
input_image = HWC3(input_image)
detected_map = apply_uniformer(resize_image(input_image, 512))
img = resize_image(input_image, 512)
H, W, C = img.shape

detected_map_seg = cv2.resize(detected_map, (W, H), interpolation=cv2.INTER_NEAREST)

control_1 = torch.from_numpy(detected_map_seg.copy()).float().cuda() / 255.0
control_1 = torch.stack([control_1 for _ in range(1)], dim=0)
control_1 = einops.rearrange(control_1, 'b h w c -> b c h w').clone()

/home/hu/MMIB/controlnet/annotator/uniformer/mmseg/models/segmentors/base.py:271: UserWarning: show==False and out_file is not specified, only result image will be returned
  warnings.warn('show==False and out_file is not specified, only '


In [7]:
img = resize_image(HWC3(input_image_1), 512)
H, W, C = img.shape

detected_map_cn = apply_canny(img, low_threshold=100, high_threshold=200)
detected_map_cn = HWC3(detected_map_cn)

control_2 = torch.from_numpy(detected_map_cn.copy()).float().cuda() / 255.0
control_2 = torch.stack([control_2 for _ in range(1)], dim=0)
control_2 = einops.rearrange(control_2, 'b h w c -> b c h w').clone()

In [8]:
seed_everything(42)

model.cuda()

n_samples = 1

prompt = ''
a_prompt = 'best quality, extremely detailed'

n_prompt = 'longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality'

cond = {"c_concat": [control_1,control_2], "c_crossattn": [model.get_learned_conditioning([prompt + ', ' + a_prompt] * n_samples)]}
un_cond = {"c_concat": [control_1,control_2], "c_crossattn": [model.get_learned_conditioning([n_prompt] * n_samples)]}
shape = (4, H // 8, W // 8)

Global seed set to 42


In [9]:
cond['c_concat']

[tensor([[[[0.0235, 0.0235, 0.0235,  ..., 0.0235, 0.0235, 0.0235],
           [0.0235, 0.0235, 0.0235,  ..., 0.0235, 0.0235, 0.0235],
           [0.0235, 0.0235, 0.0235,  ..., 0.0235, 0.0235, 0.0235],
           ...,
           [0.2392, 0.2392, 0.2392,  ..., 0.4706, 0.4706, 0.4706],
           [0.2392, 0.2392, 0.2392,  ..., 0.4706, 0.4706, 0.4706],
           [0.2392, 0.2392, 0.2392,  ..., 0.4706, 0.4706, 0.4706]],
 
          [[0.9020, 0.9020, 0.9020,  ..., 0.9020, 0.9020, 0.9020],
           [0.9020, 0.9020, 0.9020,  ..., 0.9020, 0.9020, 0.9020],
           [0.9020, 0.9020, 0.9020,  ..., 0.9020, 0.9020, 0.9020],
           ...,
           [0.9020, 0.9020, 0.9020,  ..., 0.4706, 0.4706, 0.4706],
           [0.9020, 0.9020, 0.9020,  ..., 0.4706, 0.4706, 0.4706],
           [0.9020, 0.9020, 0.9020,  ..., 0.4706, 0.4706, 0.4706]],
 
          [[0.9020, 0.9020, 0.9020,  ..., 0.9020, 0.9020, 0.9020],
           [0.9020, 0.9020, 0.9020,  ..., 0.9020, 0.9020, 0.9020],
           [0.9020, 0.90

In [43]:

seed_everything(5010)
ddim_sampler = DDIMSampler(model)
samples, intermediates = ddim_sampler.sample(50, 1,
                                                shape, cond, verbose=False, eta=0.0,
                                                unconditional_guidance_scale=9.,
                                                unconditional_conditioning=un_cond,
                                                sc_0=0.7, sc_1=0.7)


Global seed set to 5010


Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:16<00:00,  3.02it/s]


In [44]:
img = model.decode_first_stage(samples)
img = torch.clamp(img/2+0.5, 0, 1)
from torchvision.utils import save_image
save_image(control_1, 'cn.png')
save_image(control_2, 'seg.png')
save_image(img, 'mm.png')